In [1]:
import numpy as np
import multiprocessing
import ctypes
import logging

In [2]:
a = np.array([1, 2, 3, 4, 5])
b = np.ndarray(shape=a.shape, dtype=a.dtype, buffer=a)

print("a:", a)
print("b:", b)

# Modify b
b[1] = 10
print("a:", a)
print("b:", b)


a: [1 2 3 4 5]
b: [1 2 3 4 5]
a: [ 1 10  3  4  5]
b: [ 1 10  3  4  5]


In [3]:
# # Create a bytes object
# data = b'\x01\x02\x03\x04\x05\x06\x07\x08'

# # Create a NumPy array that shares the memory of the bytes object
# shared_buffer = np.frombuffer(data, dtype=np.uint8)

# print(shared_buffer)

In [4]:
num_ROIs = 7
# num_frames = 600
# num_frames = 50
num_frames = 10

# Create a shared array of 7*600=4200 int16 values
shared_buffer = multiprocessing.Array('h', num_ROIs * num_frames)

# my_obj = shared_buffer.get_obj()
# type(my_obj)

# Create a numpy array that uses the shared memory
# np_arr = np.ndarray(shape=(num_ROIs, num_frames), dtype=np.int16, buffer=arr)
# np_arr = np.frombuffer(my_obj, dtype='int16').reshape(num_ROIs, num_frames)
# np_arr = np.frombuffer(my_obj, dtype=np.int16).reshape(num_ROIs, num_frames)
# np_arr = np.frombuffer(shared_buffer.get_obj(), dtype='int16').reshape(num_ROIs, num_frames)
shared_np_arr = np.frombuffer(shared_buffer.get_obj(), dtype=np.int16).reshape(num_ROIs, num_frames)
shared_np_arr.fill(0)

print(shared_np_arr.shape)
print(shared_np_arr.dtype)
print(shared_np_arr)

(7, 10)
int16
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [5]:
# # Set the values of the NumPy array to 1, 2, 3, ..., 4200
# shared_np_arr[:] = np.arange(1, num_ROIs*num_frames+1, dtype=np.int16).reshape((num_ROIs, num_frames))

# print(shared_np_arr)

In [6]:
# Access and modify the shared array from multiple processes
def worker_process(shared_buffer, frame_num):
    print(f"Worker {frame_num} starting...")

    shared_np_arr = np.frombuffer(shared_buffer.get_obj(), dtype=np.int16).reshape(num_ROIs, num_frames)
    
    shared_np_arr[:, frame_num] = frame_num

In [7]:
processes = []

for i in range(num_frames):
    # Loop through all frames
    # print(f"Creating worker {i}...")

    p = multiprocessing.Process(target=worker_process, args=(shared_buffer, i))
    processes.append(p)
    p.start()

for p in processes:
    p.join()

# print(shared_buffer[:])

In [8]:
print(shared_np_arr)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [9]:
print(shared_buffer[:])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
# shared_arr = multiprocessing.Array(ctypes.c_double, 4200)
# # ...
# def f(i): # could be anything numpy accepts as an index such another numpy array
#     with shared_arr.get_lock(): # synchronize access
#         arr = np.frombuffer(shared_arr.get_obj()) # no data copying
#         arr[i] = -arr[i]